In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/northamericanrigcount/28-03-2025 North America Rig Count Report.xlsx
/kaggle/input/prodfile/Production.xlsm
/kaggle/input/weeklyproductionusa/BasicProductionCrudeWeekly.xlsx


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def main():
    """
    Master script that:
    1) Loads rig data (weekly/daily) and merges sub-basin production data (monthly) from 2019+.
    2) Loads separate weekly total US production data (EIA).
    3) Generates:
       - Historical rig count by basin chart
       - Historical total US production (weekly EIA) chart
       - Historical production by basin chart (excluding Prod_Total)
       - Rig vs production by basin, with dual axes (2024+)
       - Rig vs total production, dual axes (2024+)
       - A table of 3-month and 6-month rig/production deltas for every basin
       - A text page summarizing correlation findings
    4) Exports all plots & tables to a single PDF.
    """

    #==============================================================================
    # 1) LOAD & PREPARE RIG DATA
    #==============================================================================
    rig_df = pd.read_excel(
        '/kaggle/input/northamericanrigcount/28-03-2025 North America Rig Count Report.xlsx',
        sheet_name='NAM Weekly',
        usecols='A:L',
        skiprows=10,
        header=0
    )
    rig_df['US_PublishDate'] = pd.to_datetime(rig_df['US_PublishDate'])
    rig_df = rig_df[rig_df['US_PublishDate'] >= "2019-01-01"].copy()

    # Identify major basins vs. Other
    major_basins = {'Williston', 'DJ-Niobrara', 'Eagle Ford', 'Permian'}
    def unify_basin_rig(x):
        return x if x in major_basins else 'Other'
    rig_df['Basin'] = rig_df['Basin'].apply(unify_basin_rig)

    # Group & pivot to columns = Rig_<Basin>
    grouped_rig = rig_df.groupby(['US_PublishDate','Basin'], as_index=False)['Rig Count Value'].sum()
    rig_pivot = grouped_rig.pivot(index='US_PublishDate', columns='Basin', values='Rig Count Value')
    rig_pivot.columns = [f'Rig_{c}' for c in rig_pivot.columns]

    # Daily forward-fill
    full_rig_dates = pd.date_range(rig_pivot.index.min(), rig_pivot.index.max(), freq='D')
    rig_pivot = rig_pivot.reindex(full_rig_dates).sort_index().ffill()

    # Rig_Total
    rig_pivot['Rig_Total'] = rig_pivot.sum(axis=1)

    # Move index into a column
    rig_pivot = rig_pivot.rename_axis('Date').reset_index()

    #==============================================================================
    # 2) LOAD & PREPARE SUB-BASIN PRODUCTION DATA
    #==============================================================================
    prod_df = pd.read_excel(
        '/kaggle/input/prodfile/Production.xlsm',
        sheet_name='Production (Regional)',
        usecols='B:Y',
        skiprows=4,
        header=0
    )
    prod_df['Date (Month)'] = pd.to_datetime(prod_df['Date (Month)'])
    prod_df = prod_df[prod_df['Date (Month)'] >= "2019-01-01"].copy()

    basin_mapping = {
        'Bakken': 'Williston',
        'Niobrara': 'DJ-Niobrara',
        'Eagle Ford': 'Eagle Ford',
        'Permian': 'Permian'
    }
    def unify_basin_prod(x):
        return basin_mapping[x] if x in basin_mapping else 'Other'
    prod_df['Basin'] = prod_df['SBM Sub-Region'].apply(unify_basin_prod)

    # Keep first row per (month, basin) in case of duplicates
    monthly_prod = (
        prod_df
        .groupby(['Date (Month)', 'Basin'], as_index=False)['SBM Total (Mbpd)']
        .agg('first')  
    )
    # Pivot => Prod_<Basin>
    prod_pivot = monthly_prod.pivot(index='Date (Month)', columns='Basin', values='SBM Total (Mbpd)')
    prod_pivot.columns = [f'Prod_{c}' for c in prod_pivot.columns]

    # Daily reindex + forward fill
    full_prod_dates = pd.date_range(prod_pivot.index.min(), prod_pivot.index.max(), freq='D')
    prod_pivot = prod_pivot.reindex(full_prod_dates).sort_index().ffill()
    prod_pivot['Prod_Total'] = prod_pivot.sum(axis=1)

    prod_pivot = prod_pivot.rename_axis('Date').reset_index()

    #==============================================================================
    # 3) MERGE RIG & SUB-BASIN PRODUCTION
    #==============================================================================
    combined_df = pd.merge(rig_pivot, prod_pivot, on='Date', how='inner')

    #==============================================================================
    # 4) LOAD WEEKLY TOTAL US PRODUCTION (EIA) => df_production
    #==============================================================================
    df_production = pd.read_excel(
        '/kaggle/input/weeklyproductionusa/BasicProductionCrudeWeekly.xlsx',
        sheet_name='Data 1',
        usecols='A:B',
        skiprows=2,
        header=0
    )
    df_production['Date'] = pd.to_datetime(df_production['Date'])
    df_production = df_production[df_production['Date'] >= '2015-01-01'].copy()

    #==============================================================================
    # 5) BASIC ANALYSIS: CORRELATION, YOY, ETC.
    #==============================================================================
    # Correlation (same period) => Rig_Total vs. Prod_Total
    corr_same_period = combined_df[['Rig_Total','Prod_Total']].corr().iloc[0,1]

    # 3-month & 6-month forward shift
    combined_df['Prod_3mAhead'] = combined_df['Prod_Total'].shift(-90)
    combined_df['Prod_6mAhead'] = combined_df['Prod_Total'].shift(-180)
    corr_3m_lag = combined_df[['Rig_Total','Prod_3mAhead']].corr().iloc[0,1]
    corr_6m_lag = combined_df[['Rig_Total','Prod_6mAhead']].corr().iloc[0,1]

    # YoY changes
    combined_df['YoY Rig Change'] = combined_df['Rig_Total'] - combined_df['Rig_Total'].shift(365)
    combined_df['YoY Prod Change'] = combined_df['Prod_Total'] - combined_df['Prod_Total'].shift(365)

    #==============================================================================
    # 6) DELTA TABLE FOR LAST 3 AND 6 MONTHS, PER BASIN
    #==============================================================================
    def get_nearest_value(df, date, col):
        """Return the value in `col` that's on or before `date`."""
        df_sub = df[df['Date'] <= date]
        if df_sub.empty:
            return np.nan
        return df_sub.iloc[-1][col]

    # We'll produce a row for each basin in [Williston, DJ-Niobrara, Eagle Ford, Permian, Other, 'Total']
    basins_list = ['Williston','DJ-Niobrara','Eagle Ford','Permian','Other','Total']
    
    last_date = combined_df['Date'].max()
    date_3m_ago = last_date - pd.DateOffset(months=3)
    date_6m_ago = last_date - pd.DateOffset(months=6)

    delta_rows = []
    for basin in basins_list:
        # Decide which columns to use
        rig_col  = f'Rig_{basin}'  if basin != 'Total' else 'Rig_Total'
        prod_col = f'Prod_{basin}' if basin != 'Total' else 'Prod_Total'

        # Current rig / prod
        rig_now  = get_nearest_value(combined_df, last_date, rig_col)
        prod_now = get_nearest_value(combined_df, last_date, prod_col)

        # 3 months rig / prod
        rig_3m = get_nearest_value(combined_df, date_3m_ago, rig_col)
        prod_3m = get_nearest_value(combined_df, date_3m_ago, prod_col)
        rig_3m_delta = rig_now - rig_3m if pd.notnull(rig_now) and pd.notnull(rig_3m) else np.nan
        prod_3m_delta = prod_now - prod_3m if pd.notnull(prod_now) and pd.notnull(prod_3m) else np.nan

        # 6 months rig / prod
        rig_6m = get_nearest_value(combined_df, date_6m_ago, rig_col)
        prod_6m = get_nearest_value(combined_df, date_6m_ago, prod_col)
        rig_6m_delta = rig_now - rig_6m if pd.notnull(rig_now) and pd.notnull(rig_6m) else np.nan
        prod_6m_delta = prod_now - prod_6m if pd.notnull(prod_now) and pd.notnull(prod_6m) else np.nan

        delta_rows.append({
            'Basin': basin,
            '3M Rig Δ': rig_3m_delta,
            '3M Prod Δ (Mbpd)': prod_3m_delta,
            '6M Rig Δ': rig_6m_delta,
            '6M Prod Δ (Mbpd)': prod_6m_delta
        })

    delta_basin_df = pd.DataFrame(delta_rows)

    #==============================================================================
    # 7) CREATE PDF & PLOTS
    #==============================================================================
    with PdfPages('BasinProductionandRigCount.pdf') as pdf:

        #
        # 7.1) Historical Rig Count by Basin
        #
        fig1, ax1 = plt.subplots(figsize=(12,6))
        rig_cols = [c for c in rig_pivot.columns if c.startswith('Rig_')]
        for col in rig_cols:
            ax1.plot(rig_pivot['Date'], rig_pivot[col], label=col)
        ax1.set_title("Historical Rig Count by Basin (2019+)")
        ax1.set_xlabel("Date")
        ax1.set_ylabel("Rig Count")
        ax1.legend()
        pdf.savefig(fig1)
        plt.close(fig1)

        #
        # 7.2) Historical total US production (weekly EIA)
        #
        fig2, ax2 = plt.subplots(figsize=(12,6))
        ax2.plot(df_production['Date'], df_production['Weekly U.S. Field Production of Crude Oil  (Thousand Barrels per Day)'])
        ax2.set_title("Weekly US Field Production of Crude (2015+)")
        ax2.set_xlabel("Date")
        ax2.set_ylabel("Production (Thousand Barrels per Day)")
        pdf.savefig(fig2)
        plt.close(fig2)

        #
        # 7.3) Historical production by basin (EXCLUDING Prod_Total)
        #
        fig3, ax3 = plt.subplots(figsize=(12,6))
        # Grab all columns that start with 'Prod_' except 'Prod_Total'
        prod_basin_cols = [c for c in prod_pivot.columns if c.startswith('Prod_') and c != 'Prod_Total']
        for col in prod_basin_cols:
            ax3.plot(prod_pivot['Date'], prod_pivot[col], label=col)
        ax3.set_title("Historical Production by Basin (2019+)")
        ax3.set_xlabel("Date")
        ax3.set_ylabel("Production (Mbpd)")
        ax3.legend()
        pdf.savefig(fig3)
        plt.close(fig3)

        #
        # 7.4) Rig vs Production by Basin (2024+), dual axes
        #
        from_2024 = combined_df[combined_df['Date'] >= '2024-01-01'].copy()
        basins_to_plot = ['Williston','DJ-Niobrara','Eagle Ford','Permian','Other']
        for basin in basins_to_plot:
            rig_col  = f'Rig_{basin}'
            prod_col = f'Prod_{basin}'
            if rig_col not in from_2024.columns or prod_col not in from_2024.columns:
                continue

            # Make sure there's actually data
            df_basin = from_2024[['Date', rig_col, prod_col]].dropna()
            if df_basin.empty:
                continue

            fig_basin, ax_basin_1 = plt.subplots(figsize=(12,6))
            ax_basin_1.set_title(f"{basin}: Rig Count vs. Production (2024+)")
            color1 = 'tab:blue'
            ax_basin_1.set_xlabel("Date")
            ax_basin_1.set_ylabel("Rig Count", color=color1)
            ax_basin_1.plot(df_basin['Date'], df_basin[rig_col], color=color1, label='Rig Count')
            ax_basin_1.tick_params(axis='y', labelcolor=color1)

            ax_basin_2 = ax_basin_1.twinx()
            color2 = 'tab:red'
            ax_basin_2.set_ylabel("Production (Mbpd)", color=color2)
            ax_basin_2.plot(df_basin['Date'], df_basin[prod_col], color=color2, label='Production')
            ax_basin_2.tick_params(axis='y', labelcolor=color2)

            pdf.savefig(fig_basin)
            plt.close(fig_basin)

        #
        # 7.5) Rig vs Production (Total), dual axes (2024+)
        #
        df_2024_total = from_2024[['Date','Rig_Total','Prod_Total']].dropna()
        if not df_2024_total.empty:
            fig5, ax5 = plt.subplots(figsize=(12,6))
            ax5.set_title("Total Rig Count vs. Total Production (2024+)")
            ax5.set_xlabel("Date")

            color1 = 'tab:blue'
            ax5.set_ylabel("Rig Count", color=color1)
            ax5.plot(df_2024_total['Date'], df_2024_total['Rig_Total'], color=color1, label='Rig Count')
            ax5.tick_params(axis='y', labelcolor=color1)

            ax5_2 = ax5.twinx()
            color2 = 'tab:red'
            ax5_2.set_ylabel("Production (Mbpd)", color=color2)
            ax5_2.plot(df_2024_total['Date'], df_2024_total['Prod_Total'], color=color2, label='Total Production')
            ax5_2.tick_params(axis='y', labelcolor=color2)

            pdf.savefig(fig5)
            plt.close(fig5)

        #
        # 7.6) DELTA TABLE (3M, 6M) for all basins
        #
        fig_table = plt.figure(figsize=(10, 6))
        fig_table.suptitle("3-Month and 6-Month Deltas per Basin", fontsize=14, fontweight='bold')
        ax_table = fig_table.add_subplot(111)
        ax_table.axis('off')

        # Build table data
        table_data = []
        for _, row in delta_basin_df.iterrows():
            table_data.append([
                row['Basin'],
                f"{row['3M Rig Δ']:.1f}" if pd.notnull(row['3M Rig Δ']) else "",
                f"{row['3M Prod Δ (Mbpd)']:.1f}" if pd.notnull(row['3M Prod Δ (Mbpd)']) else "",
                f"{row['6M Rig Δ']:.1f}" if pd.notnull(row['6M Rig Δ']) else "",
                f"{row['6M Prod Δ (Mbpd)']:.1f}" if pd.notnull(row['6M Prod Δ (Mbpd)']) else ""
            ])

        col_labels = ["Basin", "3M Rig Δ", "3M Prod Δ (Mbpd)", "6M Rig Δ", "6M Prod Δ (Mbpd)"]
        table = ax_table.table(
            cellText=table_data,
            colLabels=col_labels,
            loc='center'
        )
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.scale(1, 1.5)  # Adjust table size as needed

        pdf.savefig(fig_table)
        plt.close(fig_table)

        #
        # 7.7) A TEXT PAGE WITH CORRELATION RESULTS & SUMMARY
        #
        summary_text = f"""
        SUMMARY OF FINDINGS

        1) Correlation (no lag) between total rig count and production: {corr_same_period:.3f}
        2) Correlation (3-month forward production) vs rig count: {corr_3m_lag:.3f}
        3) Correlation (6-month forward production) vs rig count: {corr_6m_lag:.3f}

        Observations:
        - Honestly not much correlation between rig count and basin production.
        - Thought it was interesting to witness Permian rig count decreasing currently.
        - Still production is at all time highs and only predicted to increase.
        """

        fig_text = plt.figure(figsize=(8.5, 11))
        fig_text.text(0.1, 0.9, summary_text, fontsize=12, va='top')
        pdf.savefig(fig_text)
        plt.close(fig_text)

    # End of with => "My_Oil_Analysis_Report.pdf" is finalized

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
